In [83]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) 
import numpy as np
from datetime import datetime
import streamlit as st
from db_utils import get_connection,load_query,execute_query,load_to_gsheets , load_to_gsheets_from_cell , read_from_gsheets , read_from_gsheets_area
import db_utils
import os

In [2]:
print(st.__version__)
df = pd.DataFrame(columns=['A', 'B', 'C'], data=[[1, 2, 3], [4, 5, 6]])
# load_to_gsheets(df , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'test_sheet')

1.37.1


In [3]:
years = [2023,2024,2025]
print(db_utils.__file__)

c:\Users\MKumar\AppData\Local\anaconda3\Lib\site-packages\db_utils.py


In [4]:
score_data = execute_query(load_query(r'C:\Users\MKumar\My Drive\Jupyter Workspace\MCAT Analytics DB Dev\SQL Queries/Scores_query_23_25.sql'))
print(score_data.shape)
Act_data = execute_query(load_query(r'C:\Users\MKumar\My Drive\Jupyter Workspace\MCAT Analytics DB Dev\SQL Queries/Activity_query_23_25.sql'))
print(Act_data.shape)
kfl_data = execute_query(load_query(r'C:\Users\MKumar\My Drive\Jupyter Workspace\MCAT Analytics DB Dev\SQL Queries/KFL_Query.sql'))
print(kfl_data.shape)
mcat_quiz_data = execute_query(load_query(r'C:\Users\MKumar\My Drive\Jupyter Workspace\MCAT Analytics DB Dev\SQL Queries/mcat_quiz.sql'))
print(mcat_quiz_data.shape)


Conn Open
conn closed
(140261, 14)
Conn Open
conn closed
(10178646, 16)
Conn Open
conn closed
(83515, 18)
Conn Open
conn closed
(760484, 13)


## Data Preparation 

In [5]:
score_data['sequence_title'] = score_data['sequence_name']
mcat_quiz_data['sequence_title'] = mcat_quiz_data['sequence_name']

In [6]:
kfl_data = kfl_data[~kfl_data['sequence_title'].str.contains('Practice',case=False)]
kfl_data = kfl_data[kfl_data['exam_completed']==1]
Act_data = Act_data[Act_data['status'] == 'completed']

kfl_data['scaled_score'] = pd.to_numeric(kfl_data['scaled_score'], errors='coerce')
kfl_data['date_created'] = pd.to_datetime(kfl_data['date_created'])
Act_data_cleaned = Act_data[~Act_data['sequence_title'].str.contains('AAMC Practice Exam',case=False)]

Act_data_cleaned_all = pd.concat([
   Act_data_cleaned[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','date_completed','sequence_name','sequence_title','total_scored_items_answered']],                               
   mcat_quiz_data[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','date_completed','sequence_name','sequence_title','total_scored_items_answered']],
   score_data[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','date_completed','sequence_name','sequence_title','total_scored_items_answered']]  
               ] ,ignore_index=True)
print(Act_data_cleaned_all.shape)
aamc_kfl_data =  pd.concat([score_data[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','date_completed','sequence_name','total_scored_items_answered']],
                           kfl_data[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','date_completed','sequence_name','total_scored_items_answered']] 
                           ], ignore_index=True)
print(aamc_kfl_data.shape)

Act_data_cleaned_all = Act_data_cleaned_all.dropna(subset=['enroll_exp_date'])
Act_data_cleaned_all['date_completed'] = pd.to_datetime(Act_data_cleaned_all['date_completed'])
Act_data_cleaned_all['date_created'] = pd.to_datetime(Act_data_cleaned_all['date_created'])
Act_data_cleaned_all['enroll_exp_date'] = pd.to_datetime(Act_data_cleaned_all['enroll_exp_date'] , errors='coerce')
aamc_kfl_data['date_completed'] = pd.to_datetime(aamc_kfl_data['date_completed'])
aamc_kfl_data['date_created'] = pd.to_datetime(aamc_kfl_data['date_created'])
aamc_kfl_data['enroll_exp_date'] = pd.to_datetime(aamc_kfl_data['enroll_exp_date'] , errors='coerce')

score_data_all_pt = score_data.groupby(['kbs_enrollment_id'])['sequence_name'].nunique().reset_index(name="# of PT")
score_data_pt = pd.merge(score_data ,score_data_all_pt,on="kbs_enrollment_id",how='inner',)
# score_data_pt

Act_data_cleaned_all['Activity_year'] = Act_data_cleaned_all['date_completed'].dt.year
Act_data_cleaned_all['Activity_month'] = Act_data_cleaned_all['date_completed'].dt.month
Act_data_cleaned_all['ESD_year'] = Act_data_cleaned_all['enroll_start_date'].dt.year
Act_data_cleaned_all['ESD_month'] = Act_data_cleaned_all['enroll_start_date'].dt.month
Act_data_cleaned_all['Expiry_year'] = Act_data_cleaned_all['enroll_exp_date'].dt.year
Act_data_cleaned_all['Expiry_month'] = Act_data_cleaned_all['enroll_exp_date'].dt.month

score_data_pt["enroll_exp_date"] = pd.to_datetime(score_data_pt["enroll_exp_date"], errors='coerce').astype('datetime64[ms]')
score_data_pt['Expiry_year'] = score_data_pt['enroll_exp_date'].dt.year
score_data_pt['Expiry_month'] = score_data_pt['enroll_exp_date'].dt.month

score_data_pt['Activity_year'] = score_data_pt['date_completed'].dt.year
score_data_pt['Activity_month'] = score_data_pt['date_completed'].dt.month

kfl_data['date_completed'] = pd.to_datetime(kfl_data['date_completed'])
kfl_data['Activity_year'] = kfl_data['date_completed'].dt.year
kfl_data['Activity_month'] = kfl_data['date_completed'].dt.month

aamc_kfl_data['Expiry_year'] = aamc_kfl_data['enroll_exp_date'].dt.year
aamc_kfl_data['Expiry_month'] = aamc_kfl_data['enroll_exp_date'].dt.month
aamc_kfl_data['Activity_year'] = aamc_kfl_data['date_completed'].dt.year
aamc_kfl_data['Activity_month'] = aamc_kfl_data['date_completed'].dt.month

Act_data_cleaned_all['ESD_Quarter'] = 'Q'+Act_data_cleaned_all['enroll_start_date'].dt.quarter.astype(str)
Act_data_cleaned_all['Expiry_Quarter'] = 'Q'+Act_data_cleaned_all['enroll_exp_date'].dt.quarter.astype(str)
Act_data_cleaned_all['Activity_Quarter'] = 'Q'+ Act_data_cleaned_all['date_completed'].dt.quarter.astype(str)
aamc_kfl_data['Activity_Quarter'] = 'Q'+ aamc_kfl_data['date_completed'].dt.quarter.astype(str)
aamc_kfl_data['Expiry_Quarter'] = 'Q'+ aamc_kfl_data['enroll_exp_date'].dt.quarter.astype(str)
score_data_pt['Activity_Quarter'] = 'Q' + score_data_pt['date_completed'].dt.quarter.astype(str)
score_data_pt['Expiry_Quarter'] = 'Q' + score_data_pt['enroll_exp_date'].dt.quarter.astype(str)
kfl_data['Activity_Quarter'] = 'Q' + kfl_data['date_completed'].dt.quarter.astype(str)
score_data_pt['Expiry_Quarter'] = 'Q' + score_data_pt['enroll_exp_date'].dt.quarter.astype(str)



(10962282, 9)
(223705, 8)


In [7]:
prod_code_df = pd.DataFrame(Act_data_cleaned_all['product_code'].unique(), columns=['product_code'])
load_to_gsheets(prod_code_df , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Product Code' )

## KPIs on Previous Month

In [65]:
today = datetime.today()
prev_month = today.month - 1 if today.month > 1 else 12
prev_prev_month = today.month - 2 if today.month > 2 else 12
all_prod = Act_data_cleaned_all['product_code'].unique()
print(prev_month)

9


### Score Gain Previous Month

In [84]:
expiry_24_ap_score = score_data_pt[(score_data_pt["Expiry_year"] == 2024) & (score_data_pt["Expiry_month"] == prev_month)]
expiry_25_ap_score = score_data_pt[(score_data_pt["Expiry_year"] == 2025) & (score_data_pt["Expiry_month"] == prev_month)]

print(expiry_24_ap_score['kbs_enrollment_id'].nunique() , expiry_25_ap_score['kbs_enrollment_id'].nunique())

expiry_24_ap_score_2 = expiry_24_ap_score[expiry_24_ap_score["# of PT"] >=2]
expiry_25_ap_score_2 = expiry_25_ap_score[expiry_25_ap_score["# of PT"] >=2]
print(expiry_24_ap_score_2['kbs_enrollment_id'].nunique() , expiry_25_ap_score_2['kbs_enrollment_id'].nunique())

first_score_24 = expiry_24_ap_score_2[expiry_24_ap_score_2['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
first_score_24 = first_score_24.loc[ (first_score_24.groupby(['kbs_enrollment_id'])['date_created'].idxmin())]

max_score_24 = expiry_24_ap_score_2[expiry_24_ap_score_2['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
max_score_24 =   max_score_24.loc[max_score_24.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]

first_score_25 = expiry_25_ap_score_2[expiry_25_ap_score_2['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
first_score_25 = first_score_25.loc[ (first_score_25.groupby(['kbs_enrollment_id'])['date_created'].idxmin())]

max_score_25 = expiry_25_ap_score_2[expiry_25_ap_score_2['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
max_score_25 =   max_score_25.loc[max_score_25.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]

score_gain_24 = pd.merge(first_score_24[['kbs_enrollment_id','product_code','scaled_score']] , 
                        max_score_24[['kbs_enrollment_id','scaled_score']] , on = 'kbs_enrollment_id' , how='inner' , suffixes=('_first','_max'))
score_gain_24['diff_score'] = score_gain_24['scaled_score_max'] - score_gain_24['scaled_score_first']

score_gain_25 = pd.merge(first_score_25[['kbs_enrollment_id','product_code','scaled_score']] , 
                        max_score_25[['kbs_enrollment_id','scaled_score']] , on = 'kbs_enrollment_id' , how='inner' , suffixes=('_first','_max'))
score_gain_25['diff_score'] = score_gain_25['scaled_score_max'] - score_gain_25['scaled_score_first']

print("24 Score Gain " , score_gain_24['diff_score'].sum() / score_gain_24['kbs_enrollment_id'].nunique() )
print("25 Score Gain " , score_gain_25['diff_score'].sum() / score_gain_25['kbs_enrollment_id'].nunique() )
# score_gain_25['diff_score'].value_counts()

all_product_codes = Act_data_cleaned_all['product_code'].unique()

def get_score_gain_all_avg(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_score_diff = 0
            total_eid = 0
        else:
            total_score_diff = temp[col_name].sum()
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_score_diff, f'total_eid_{suffix}': total_eid})
    final_score_gain = pd.DataFrame(results)
    return final_score_gain
final_score_gain_24 = get_score_gain_all_avg(score_gain_24 , 'diff_score' , '24')
final_score_gain_25 = get_score_gain_all_avg(score_gain_25 , 'diff_score' , '25')

final_score_gain_25_24 = pd.merge(
    final_score_gain_24 , final_score_gain_25 , on='product_code' , how='inner'
)
final_score_gain_25_24['Parameter'] = 'Score Gain'
load_to_gsheets_from_cell(final_score_gain_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet' , 'A2')
print(final_score_gain_25_24['total_diff_score_24'].sum()/final_score_gain_25_24['total_eid_24'].sum())
# final_score_gain_25_24



902 1122
655 786
24 Score Gain  10.21462639109698
25 Score Gain  10.567460317460318
10.21462639109698


## Average Active User and activity Per User

In [85]:
act_user_24 = Act_data_cleaned_all[(Act_data_cleaned_all['Activity_month'] == prev_month) & (Act_data_cleaned_all['Activity_year'] == 2024)]
act_user_25 = Act_data_cleaned_all[(Act_data_cleaned_all['Activity_month'] == prev_month) & (Act_data_cleaned_all['Activity_year'] == 2025)]
print("Sequence_name Per User 24 " , act_user_24['sequence_name'].count() /act_user_24['kbs_enrollment_id'].nunique() )
print("Sequence_name Per User 25 " , act_user_25['sequence_name'].count() /act_user_25['kbs_enrollment_id'].nunique() )
print("Active User 24"  , act_user_24['kbs_enrollment_id'].nunique())
print("Active User 25"  , act_user_25['kbs_enrollment_id'].nunique())

def get_avg_act(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_act = 0
            total_eid = 0
        else:
            total_act = temp[col_name].count()
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_act, f'total_eid_{suffix}': total_eid})
    final_avg_act = pd.DataFrame(results)
    return final_avg_act
final_avg_act_24 = get_avg_act(act_user_24 , 'sequence_name' , '24')
final_avg_act_25 = get_avg_act(act_user_25 , 'sequence_name' , '25')
final_avg_act_25_24 = pd.merge(
    final_avg_act_24 , final_avg_act_25 , on='product_code' , how='inner'
)
final_avg_act_25_24['Parameter'] = 'Avg Activity / User'
print(final_avg_act_25_24['total_sequence_name_25'].sum()/final_avg_act_25_24['total_eid_25'].sum())
load_to_gsheets_from_cell(final_avg_act_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet' , 'H2')

def get_active_user(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_eid = 0
        else:
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_eid})
    final_active_user = pd.DataFrame(results)
    return final_active_user
final_active_user_24 = get_active_user(act_user_24 , 'active_user' , '24')
final_active_user_25 = get_active_user(act_user_25 , 'active_user' , '25')
final_active_user_25_24 = pd.merge(
    final_active_user_24 , final_active_user_25 , on='product_code' , how='inner'
)
final_active_user_25_24['Parameter'] = 'Active Users'
print(final_active_user_25_24['total_active_user_25'].sum() , final_active_user_25_24['total_active_user_24'].sum())
load_to_gsheets_from_cell(final_active_user_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet' , 'O2')

Sequence_name Per User 24  35.00425696594427
Sequence_name Per User 25  42.11219428766889
Active User 24 5168
Active User 25 5847
42.11219428766889
5847 5168


## Avg Questions Answered

In [86]:
print(act_user_24['total_scored_items_answered'].sum() /act_user_24['kbs_enrollment_id'].nunique() )
print(act_user_25['total_scored_items_answered'].sum() /act_user_25['kbs_enrollment_id'].nunique() )

avg_ques_ans_24 = get_score_gain_all_avg(act_user_24 , 'total_scored_items_answered' , '24')
avg_ques_ans_25 = get_score_gain_all_avg(act_user_25 , 'total_scored_items_answered' , '25')
final_avg_ques_ans_25_24 = pd.merge(
    avg_ques_ans_24 , avg_ques_ans_25 , on='product_code' , how='inner'
)
print(final_avg_ques_ans_25_24['total_total_scored_items_answered_24'].sum()/final_avg_ques_ans_25_24['total_eid_24'].sum())
print(final_avg_ques_ans_25_24['total_total_scored_items_answered_25'].sum()/final_avg_ques_ans_25_24['total_eid_25'].sum())
final_avg_ques_ans_25_24['Parameter'] = 'Avg Questions Answered / User'
load_to_gsheets_from_cell(final_avg_ques_ans_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet' , 'T2')

test_num_24 = aamc_kfl_data[(aamc_kfl_data['Activity_year'] == 2024) & (aamc_kfl_data['Activity_month'] == prev_month)]
test_num_25 = aamc_kfl_data[(aamc_kfl_data['Activity_year'] == 2025) & (aamc_kfl_data['Activity_month'] == prev_month)]
print("Avg Test/User 24 " , test_num_24['sequence_name'].count() / test_num_24['kbs_enrollment_id'].nunique())
print("Avg Test/User 25 " , test_num_25['sequence_name'].count() / test_num_25['kbs_enrollment_id'].nunique())

avg_test_24 = get_avg_act(test_num_24 , 'sequence_name' , '24')
avg_test_25 = get_avg_act(test_num_25 , 'sequence_name' , '25')
final_test_25_24 = pd.merge(
    avg_test_24 , avg_test_25 , on='product_code' , how='inner'
)
final_test_25_24['Parameter'] = 'Avg Tests / User'
print(final_test_25_24['total_sequence_name_25'].sum()/final_test_25_24['total_eid_25'].sum())
print(final_test_25_24['total_sequence_name_24'].sum()/final_test_25_24['total_eid_24'].sum())
load_to_gsheets_from_cell(final_test_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet' , 'AA2')



255.59075077399382
214.9013169146571
255.59075077399382
214.9013169146571
Avg Test/User 24  1.3442964196502913
Avg Test/User 25  1.2643504531722054
1.2643504531722054
1.3442964196502913


## KPIs till Previous Month

In [11]:
expiry_24_ap_score = score_data_pt[(score_data_pt["Expiry_year"] == 2024) & (score_data_pt["Expiry_month"] <= prev_month)]
expiry_25_ap_score = score_data_pt[(score_data_pt["Expiry_year"] == 2025) & (score_data_pt["Expiry_month"] <= prev_month)]

print(expiry_24_ap_score['kbs_enrollment_id'].nunique() , expiry_25_ap_score['kbs_enrollment_id'].nunique())

expiry_24_ap_score_2 = expiry_24_ap_score[expiry_24_ap_score["# of PT"] >=2]
expiry_25_ap_score_2 = expiry_25_ap_score[expiry_25_ap_score["# of PT"] >=2]
print(expiry_24_ap_score_2['kbs_enrollment_id'].nunique() , expiry_25_ap_score_2['kbs_enrollment_id'].nunique())

first_score_24 = expiry_24_ap_score_2[expiry_24_ap_score_2['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
first_score_24 = first_score_24.loc[ (first_score_24.groupby(['kbs_enrollment_id'])['date_created'].idxmin())]

max_score_24 = expiry_24_ap_score_2[expiry_24_ap_score_2['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
max_score_24 =   max_score_24.loc[max_score_24.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]

first_score_25 = expiry_25_ap_score_2[expiry_25_ap_score_2['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
first_score_25 = first_score_25.loc[ (first_score_25.groupby(['kbs_enrollment_id'])['date_created'].idxmin())]

max_score_25 = expiry_25_ap_score_2[expiry_25_ap_score_2['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
max_score_25 =   max_score_25.loc[max_score_25.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]

score_gain_24 = pd.merge(first_score_24[['kbs_enrollment_id','product_code','scaled_score']] , 
                        max_score_24[['kbs_enrollment_id','scaled_score']] , on = 'kbs_enrollment_id' , how='inner' , suffixes=('_first','_max'))
score_gain_24['diff_score'] = score_gain_24['scaled_score_max'] - score_gain_24['scaled_score_first']

score_gain_25 = pd.merge(first_score_25[['kbs_enrollment_id','product_code','scaled_score']] , 
                        max_score_25[['kbs_enrollment_id','scaled_score']] , on = 'kbs_enrollment_id' , how='inner' , suffixes=('_first','_max'))
score_gain_25['diff_score'] = score_gain_25['scaled_score_max'] - score_gain_25['scaled_score_first']

print("24 Score Gain " , score_gain_24['diff_score'].sum() / score_gain_24['kbs_enrollment_id'].nunique() )
print("25 Score Gain " , score_gain_25['diff_score'].sum() / score_gain_25['kbs_enrollment_id'].nunique() )
# score_gain_25['diff_score'].value_counts()

all_product_codes = Act_data_cleaned_all['product_code'].unique()

def get_score_gain_all_avg(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_score_diff = 0
            total_eid = 0
        else:
            total_score_diff = temp[col_name].sum()
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_score_diff, f'total_eid_{suffix}': total_eid})
    final_score_gain = pd.DataFrame(results)
    return final_score_gain
final_score_gain_24 = get_score_gain_all_avg(score_gain_24 , 'diff_score' , '24')
final_score_gain_25 = get_score_gain_all_avg(score_gain_25 , 'diff_score' , '25')

final_score_gain_25_24 = pd.merge(
    final_score_gain_24 , final_score_gain_25 , on='product_code' , how='inner'
)
final_score_gain_25_24['Parameter'] = 'Score Gain'
load_to_gsheets_from_cell(final_score_gain_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet_tll_PM' , 'A2')
print(final_score_gain_25_24['total_diff_score_24'].sum()/final_score_gain_25_24['total_eid_24'].sum())
# final_score_gain_25_24



8472 9794
6064 6957
24 Score Gain  10.723011120615912
25 Score Gain  10.764558084540349
10.723011120615912


In [80]:
act_user_24 = Act_data_cleaned_all[(Act_data_cleaned_all['Activity_month'] <= prev_month) & (Act_data_cleaned_all['Activity_year'] == 2024)]
act_user_25 = Act_data_cleaned_all[(Act_data_cleaned_all['Activity_month'] <= prev_month) & (Act_data_cleaned_all['Activity_year'] == 2025)]
print("Sequence_name Per User 24 " , act_user_24['sequence_name'].count() /act_user_24['kbs_enrollment_id'].nunique() )
print("Sequence_name Per User 25 " , act_user_25['sequence_name'].count() /act_user_25['kbs_enrollment_id'].nunique() )
print("Active User 24"  , act_user_24['kbs_enrollment_id'].nunique())
print("Active User 25"  , act_user_25['kbs_enrollment_id'].nunique())

def get_avg_act(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_act = 0
            total_eid = 0
        else:
            total_act = temp[col_name].count()
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_act, f'total_eid_{suffix}': total_eid})
    final_avg_act = pd.DataFrame(results)
    return final_avg_act
final_avg_act_24 = get_avg_act(act_user_24 , 'sequence_name' , '24')
final_avg_act_25 = get_avg_act(act_user_25 , 'sequence_name' , '25')
final_avg_act_25_24 = pd.merge(
    final_avg_act_24 , final_avg_act_25 , on='product_code' , how='inner'
)
final_avg_act_25_24['Parameter'] = 'Avg Activity / User'
print(final_avg_act_25_24['total_sequence_name_25'].sum()/final_avg_act_25_24['total_eid_25'].sum())
load_to_gsheets_from_cell(final_avg_act_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet_tll_PM' , 'H2')

def get_active_user(df_in , col_name , suffix):    
    results = []
    for pcode in all_product_codes:
        temp = df_in[df_in['product_code'] == pcode]
        if temp.shape[0] == 0:
            total_eid = 0
        else:
            total_eid = temp['kbs_enrollment_id'].nunique()
        results.append({'product_code': pcode, f'total_{col_name}_{suffix}': total_eid})
    final_active_user = pd.DataFrame(results)
    return final_active_user
final_active_user_24 = get_active_user(act_user_24 , 'active_user' , '24')
final_active_user_25 = get_active_user(act_user_25 , 'active_user' , '25')
final_active_user_25_24 = pd.merge(
    final_active_user_24 , final_active_user_25 , on='product_code' , how='inner'
)
final_active_user_25_24['Parameter'] = 'Active Users'
print(final_active_user_25_24['total_active_user_25'].sum() , final_active_user_25_24['total_active_user_24'].sum())
load_to_gsheets_from_cell(final_active_user_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet_tll_PM' , 'O2')

Sequence_name Per User 24  151.29115555097414
Sequence_name Per User 25  167.51355142301253
Active User 24 19402
Active User 25 22101
167.51355142301253
22101 19402


In [82]:
print(act_user_24['total_scored_items_answered'].sum() /act_user_24['kbs_enrollment_id'].nunique() )
print(act_user_25['total_scored_items_answered'].sum() /act_user_25['kbs_enrollment_id'].nunique() )

avg_ques_ans_24 = get_score_gain_all_avg(act_user_24 , 'total_scored_items_answered' , '24')
avg_ques_ans_25 = get_score_gain_all_avg(act_user_25 , 'total_scored_items_answered' , '25')
final_avg_ques_ans_25_24 = pd.merge(
    avg_ques_ans_24 , avg_ques_ans_25 , on='product_code' , how='inner'
)
print(final_avg_ques_ans_25_24['total_total_scored_items_answered_24'].sum()/final_avg_ques_ans_25_24['total_eid_24'].sum())
print(final_avg_ques_ans_25_24['total_total_scored_items_answered_25'].sum()/final_avg_ques_ans_25_24['total_eid_25'].sum())
final_avg_ques_ans_25_24['Parameter'] = 'Avg Questions Answered / User'
load_to_gsheets_from_cell(final_avg_ques_ans_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet_tll_PM' , 'T2')

test_num_24 = aamc_kfl_data[(aamc_kfl_data['Activity_year'] == 2024) & (aamc_kfl_data['Activity_month'] <= prev_month)]
test_num_25 = aamc_kfl_data[(aamc_kfl_data['Activity_year'] == 2025) & (aamc_kfl_data['Activity_month'] <= prev_month)]
print("Avg Test/User 24 " , test_num_24['sequence_name'].count() / test_num_24['kbs_enrollment_id'].nunique())
print("Avg Test/User 25 " , test_num_25['sequence_name'].count() / test_num_25['kbs_enrollment_id'].nunique())

avg_test_24 = get_avg_act(test_num_24 , 'sequence_name' , '24')
avg_test_25 = get_avg_act(test_num_25 , 'sequence_name' , '25')
final_test_25_24 = pd.merge(
    avg_test_24 , avg_test_25 , on='product_code' , how='inner'
)
final_test_25_24['Parameter'] = 'Avg Tests / User'
print(final_test_25_24['total_sequence_name_25'].sum()/final_test_25_24['total_eid_25'].sum())
print(final_test_25_24['total_sequence_name_24'].sum()/final_test_25_24['total_eid_24'].sum())
load_to_gsheets_from_cell(final_test_25_24 , '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'KPIs_sheet_tll_PM' , 'AA2')



1319.899701061746
1326.9308176100628
1319.899701061746
1326.9308176100628
Avg Test/User 24  4.301935988527475
Avg Test/User 25  4.467485365232884
4.467485365232884
4.301935988527475


## Score Gain by Expiry Month

In [14]:
score_data_pt['scaled_score'] = score_data_pt['scaled_score'].astype(int)
score_data_all_gt_2 = score_data_pt[score_data_pt["# of PT"] >=2]

score_data_first_all = score_data_all_gt_2[score_data_all_gt_2['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
score_data_first_all = score_data_first_all.loc[ (score_data_first_all.groupby(['kbs_enrollment_id'])['date_created'].idxmin())]

score_data_max_all = score_data_all_gt_2[score_data_all_gt_2['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
score_data_max_all = score_data_max_all.loc[score_data_max_all.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]

merge_all_first_max_score = pd.merge(
    score_data_first_all[['kbs_enrollment_id','product_code','enroll_start_date','enroll_exp_date','date_created','scaled_score','total_scored_items_answered']]
    , score_data_max_all[['kbs_enrollment_id','date_created','scaled_score','total_scored_items_answered']]
    , on='kbs_enrollment_id'
    , how='inner'
    , suffixes=('_first','_max')
)
merge_all_first_max_score['score_gain'] = merge_all_first_max_score['scaled_score_max'] - merge_all_first_max_score['scaled_score_first']

merge_all_first_max_score['Expiry_year'] = merge_all_first_max_score['enroll_exp_date'].dt.year
merge_all_first_max_score['Expiry_month'] = merge_all_first_max_score['enroll_exp_date'].dt.month

In [15]:
merge_all_first_max_score = merge_all_first_max_score[merge_all_first_max_score['Expiry_year'].isin(years)]

def get_agg_p_code_yr_month(df_in , month_col,year_col , agg_col):
    agg_df = (
        df_in.groupby(['product_code', month_col, year_col])
        .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), score_gain=('score_gain', 'sum'))
        .reset_index()
    )
    pivot_df = agg_df.pivot_table(
        index=['product_code', month_col],
        columns=year_col,
        values=['kbs_enrollment_id', agg_col],
        fill_value=0
    )
    pivot_df.columns = [f"{col1}_{col2}" for col1, col2 in pivot_df.columns]
    final_df = pivot_df.reset_index()

    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(merge_all_first_max_score[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])

    final_df = (
        final_df.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    final_df = final_df.reindex(columns=cols)
    print(final_df['product_code'].nunique())
    final_df[month_col] = pd.to_datetime(final_df[month_col], format='%m').dt.strftime('%b')
    return final_df

score_gain_metric = get_agg_p_code_yr_month(merge_all_first_max_score , 'Expiry_month' , 'Expiry_year' , 'score_gain')
load_to_gsheets(score_gain_metric, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Score_Gain_Sheet' )

25


## Detailed Score Breakdown (Ref to score Gain)

In [16]:
score_data_all_score_gain = score_data_all_gt_2[(score_data_all_gt_2['kbs_enrollment_id'].isin(merge_all_first_max_score['kbs_enrollment_id'].unique()))
                                                & (score_data_all_gt_2['Expiry_year'].isin(years))]
score_data_all_score_gain['Expiry_month'] = score_data_all_score_gain['Expiry_month'].astype(int)
score_data_all_score_gain['Expiry_year'] = score_data_all_score_gain['Expiry_year'].astype(int)
def get_first_score_all(month_col, agg_col):
    first_score_all = score_data_all_score_gain[score_data_all_score_gain['sequence_name'] == "MCAT Official Prep Practice Exam 1"]
    first_score_all = first_score_all.loc[first_score_all.groupby(['kbs_enrollment_id'])['date_created'].idxmin()]
    first_score_all = (
            first_score_all.groupby(['product_code', month_col, 'Expiry_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), scaled_score=('scaled_score', 'sum'))
            .reset_index()
        )
    first_score_all_pivot = first_score_all.pivot_table(
            index=['product_code', month_col],
            columns='Expiry_year',
            values=['kbs_enrollment_id', 'scaled_score'],
            fill_value=0
        )

    first_score_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in first_score_all_pivot.columns]
    first_score_all_pivot = first_score_all_pivot.reset_index()
    first_score_all_pivot.columns.name = None  # Remove the columns name
    first_score_all_pivot['Expiry_month'] = first_score_all_pivot['Expiry_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(merge_all_first_max_score[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])

    first_score_all_pivot = (
        first_score_all_pivot.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    first_score_all_pivot = first_score_all_pivot.reindex(columns=cols)
    # print(first_score_all_pivot['product_code'].nunique())
    first_score_all_pivot[month_col] = pd.to_datetime(first_score_all_pivot[month_col], format='%m').dt.strftime('%b')
    col_rename = []
    for col in first_score_all_pivot.columns:
        if col in ['product_code', 'Expiry_month']:
            col_rename.append(col)
        else:
            col_rename.append(f'First_score_{col}')
    first_score_all_pivot.columns = col_rename
    return first_score_all_pivot
first_score_all = get_first_score_all('Expiry_month' , 'scaled_score')
# first_score_all

def get_Max_score_all(month_col, agg_col):
    max_score_all = score_data_all_score_gain[score_data_all_score_gain['sequence_name'] != "MCAT Official Prep Practice Exam 1"]
    max_score_all = max_score_all.loc[max_score_all.groupby(['kbs_enrollment_id'])['scaled_score'].idxmax()]
    max_score_all = (
            max_score_all.groupby(['product_code', month_col, 'Expiry_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), scaled_score=('scaled_score', 'sum'))
            .reset_index()
        )
    max_score_all_pivot = max_score_all.pivot_table(
            index=['product_code', month_col],
            columns='Expiry_year',
            values=['kbs_enrollment_id', 'scaled_score'],
            fill_value=0
        )

    max_score_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in max_score_all_pivot.columns]
    max_score_all_pivot = max_score_all_pivot.reset_index()
    max_score_all_pivot.columns.name = None  # Remove the columns name
    max_score_all_pivot['Expiry_month'] = max_score_all_pivot['Expiry_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(merge_all_first_max_score[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])

    max_score_all_pivot = (
        max_score_all_pivot.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    max_score_all_pivot = max_score_all_pivot.reindex(columns=cols)
    # print(max_score_all_pivot['product_code'].nunique())
    max_score_all_pivot[month_col] = pd.to_datetime(max_score_all_pivot[month_col], format='%m').dt.strftime('%b')
    col_rename = []
    for col in max_score_all_pivot.columns:
        if col in ['product_code', 'Expiry_month']:
            col_rename.append(col)
        else:
            col_rename.append(f'Max_score_{col}')
    max_score_all_pivot.columns = col_rename
    return max_score_all_pivot
max_score_all = get_Max_score_all('Expiry_month' , 'scaled_score')
# max_score_all


def get_Latest_score_all(month_col, agg_col):
    Latest_score_all = score_data_all_score_gain.loc[score_data_all_score_gain.groupby(['kbs_enrollment_id'])['date_created'].idxmax()]
    Latest_score_all = (
            Latest_score_all.groupby(['product_code', month_col, 'Expiry_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), scaled_score=('scaled_score', 'sum'))
            .reset_index()
        )
    Latest_score_all_pivot = Latest_score_all.pivot_table(
            index=['product_code', month_col],
            columns='Expiry_year',
            values=['kbs_enrollment_id', 'scaled_score'],
            fill_value=0
        )

    Latest_score_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in Latest_score_all_pivot.columns]
    Latest_score_all_pivot = Latest_score_all_pivot.reset_index()
    Latest_score_all_pivot.columns.name = None  # Remove the columns name
    Latest_score_all_pivot['Expiry_month'] = Latest_score_all_pivot['Expiry_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(merge_all_first_max_score[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])

    Latest_score_all_pivot = (
        Latest_score_all_pivot.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    Latest_score_all_pivot = Latest_score_all_pivot.reindex(columns=cols)
    # print(Latest_score_all_pivot['product_code'].nunique())
    Latest_score_all_pivot[month_col] = pd.to_datetime(Latest_score_all_pivot[month_col], format='%m').dt.strftime('%b')
    col_rename = []
    for col in Latest_score_all_pivot.columns:
        if col in ['product_code', 'Expiry_month']:
            col_rename.append(col)
        else:
            col_rename.append(f'Latest_score_{col}')
    Latest_score_all_pivot.columns = col_rename
    return Latest_score_all_pivot
Latest_score_all = get_Latest_score_all('Expiry_month' , 'scaled_score')
# Latest_score_all

detailed_sc_br_all = pd.concat([first_score_all , max_score_all, Latest_score_all] , axis=1)
load_to_gsheets(detailed_sc_br_all, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Detailed Score Breakdown_Metrics')


## Monthly Full length Engagement

In [28]:
def get_aamc_pivots(month_col, agg_col,seq_name,col_prefix):
    data_aamc = score_data_pt[score_data_pt['sequence_name'] == seq_name]
    data_aamc = data_aamc.loc[data_aamc.groupby(['kbs_enrollment_id'])['date_created'].idxmin()]
    data_aamc_all = (
            data_aamc.groupby(['product_code', month_col, 'Activity_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), scaled_score=('scaled_score', 'sum'))
            .reset_index()
        )   
    data_aamc_all_pivot = data_aamc_all.pivot_table(
            index=['product_code', month_col],
            columns='Activity_year',
            values=['kbs_enrollment_id', 'scaled_score'],
            fill_value=0
        )
    data_aamc_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in data_aamc_all_pivot.columns]
    data_aamc_all_pivot = data_aamc_all_pivot.reset_index()
    data_aamc_all_pivot.columns.name = None  # Remove the columns name
    data_aamc_all_pivot['Activity_month'] = data_aamc_all_pivot['Activity_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(data_aamc_all_pivot[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])

    data_aamc_all_pivot = (
        data_aamc_all_pivot.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    data_aamc_all_pivot = data_aamc_all_pivot.reindex(columns=cols)
    # print(first_score_all_pivot['product_code'].nunique())
    data_aamc_all_pivot[month_col] = pd.to_datetime(data_aamc_all_pivot[month_col], format='%m').dt.strftime('%b')
    col_rename = []
    for col in data_aamc_all_pivot.columns:
        if col in ['product_code', 'Activity_month']:
            col_rename.append(col)
        else:
            col_rename.append(f'{col_prefix}_score_{col}')
    data_aamc_all_pivot.columns = col_rename
    return data_aamc_all_pivot


def get_KFL_pivots(month_col, agg_col , seq_name , col_prefix):
    data_kfl1 = kfl_data[kfl_data['sequence_name'].isin(seq_name)]
    data_kfl1 = data_kfl1.loc[data_kfl1.groupby(['kbs_enrollment_id'])['date_created'].idxmin()]
    data_kfl1_all = (
            data_kfl1.groupby(['product_code', month_col, 'Activity_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), scaled_score=('scaled_score', 'sum'))
            .reset_index()
        )   
    data_kfl1_all_pivot = data_kfl1_all.pivot_table(
            index=['product_code', month_col],
            columns='Activity_year',
            values=['kbs_enrollment_id', 'scaled_score'],
            fill_value=0
        )
    data_kfl1_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in data_kfl1_all_pivot.columns]
    data_kfl1_all_pivot = data_kfl1_all_pivot.reset_index()
    data_kfl1_all_pivot.columns.name = None  # Remove the columns name
    data_kfl1_all_pivot['Activity_month'] = data_kfl1_all_pivot['Activity_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(data_kfl1_all_pivot[month_col].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', month_col])
    data_kfl1_all_pivot = (
        data_kfl1_all_pivot.set_index(['product_code', month_col])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    cols = ['product_code', month_col]
    for y in years:
        cols += [f"kbs_enrollment_id_{y}", f"{agg_col}_{y}"]
    # Reorder columns to desired sequence
    data_kfl1_all_pivot = data_kfl1_all_pivot.reindex(columns=cols)
    # print(first_score_all_pivot['product_code'].nunique())
    data_kfl1_all_pivot[month_col] = pd.to_datetime(data_kfl1_all_pivot[month_col], format='%m').dt.strftime('%b')
    col_rename = []
    for col in data_kfl1_all_pivot.columns:
        if col in ['product_code', 'Activity_month']:
            col_rename.append(col)
        else:
            col_rename.append(f'{col_prefix}_score_{col}')
    data_kfl1_all_pivot.columns = col_rename
    return data_kfl1_all_pivot

aamc1_data = get_aamc_pivots('Activity_month' , 'scaled_score','MCAT Official Prep Practice Exam 1','AAMC1')
aamc2_data = get_aamc_pivots('Activity_month' , 'scaled_score','MCAT Official Prep Practice Exam 2','AAMC2')
aamc3_data = get_aamc_pivots('Activity_month' , 'scaled_score','MCAT Official Prep Practice Exam 3','AAMC3')
aamc4_data = get_aamc_pivots('Activity_month' , 'scaled_score','MCAT Official Prep Practice Exam 4','AAMC4')
aamc5_data = get_aamc_pivots('Activity_month' , 'scaled_score','MCAT Official Prep Free Practice Exam (Scored)','AAMC5')
kfl1_act_month = get_KFL_pivots('Activity_month' , 'scaled_score', ['mcat22-fl-1-full','mcat-fl-1-1b6e4'],'KFL1')
kfl2_act_month = get_KFL_pivots('Activity_month' , 'scaled_score', ['mcat22-fl-2-full','mcat-fl-2-1b6e5'],'KFL2')
kfl3_act_month = get_KFL_pivots('Activity_month' , 'scaled_score', ['mcat22-fl-3-full','mcat-fl-3-1b6e6'],'KFL3')
monthly_FL_eng = pd.concat([aamc1_data,kfl1_act_month , kfl2_act_month , kfl3_act_month,aamc2_data , aamc3_data,aamc4_data,aamc5_data] , axis=1)
load_to_gsheets(monthly_FL_eng, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Monthly Full Length Engagement_metrics')

## Monthwise Activity / Ques / practice Test per active user

In [ ]:
Act_data_cleaned_all_req = Act_data_cleaned_all[Act_data_cleaned_all['Activity_year'].isin(years)]
aamc_kfl_data = aamc_kfl_data[aamc_kfl_data['Activity_year'].isin(years)]
def get_monthwise_act():
    act_agg_all = (
            Act_data_cleaned_all_req.groupby(['product_code', 'Activity_month', 'Activity_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), sequence_title=('sequence_title', 'count'))
            .reset_index()
        )
    act_agg_all_pivot = act_agg_all.pivot_table(
            index=['product_code', 'Activity_month'],
            columns='Activity_year',
            values=['kbs_enrollment_id', 'sequence_title'],
            fill_value=0
        )
    act_agg_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in act_agg_all_pivot.columns]
    act_agg_all_pivot = act_agg_all_pivot.reset_index()
    act_agg_all_pivot.columns.name = None  # Remove the columns name
    act_agg_all_pivot['Activity_month'] = act_agg_all_pivot['Activity_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(act_agg_all_pivot['Activity_month'].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', 'Activity_month'])
    act_agg_all_pivot = (
        act_agg_all_pivot.set_index(['product_code', 'Activity_month'])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    for c in  act_agg_all_pivot.columns:
        if c not in ['product_code', 'Activity_month']:
            act_agg_all_pivot.rename(columns={c: f'Avg_Activity_{c}'}, inplace=True)
    act_agg_all_pivot['Activity_month'] = pd.to_datetime(act_agg_all_pivot['Activity_month'], format='%m').dt.strftime('%b')
    return act_agg_all_pivot
monthwise_act = get_monthwise_act()
def get_monthwise_question():
    ques_agg_all = (
            Act_data_cleaned_all_req.groupby(['product_code', 'Activity_month', 'Activity_year' ])
            .agg(kbs_enrollment_id=('kbs_enrollment_id', 'nunique'), total_scored_items_answered=('total_scored_items_answered', 'sum'))
            .reset_index()
        )
    ques_agg_all_pivot = ques_agg_all.pivot_table(
            index=['product_code', 'Activity_month'],
            columns='Activity_year',
            values=['kbs_enrollment_id', 'total_scored_items_answered'],
            fill_value=0
        )
    ques_agg_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in ques_agg_all_pivot.columns]
    ques_agg_all_pivot = ques_agg_all_pivot.reset_index()
    ques_agg_all_pivot.columns.name = None  # Remove the columns name
    ques_agg_all_pivot['Activity_month'] = ques_agg_all_pivot['Activity_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = Act_data_cleaned_all['product_code'].unique()
    all_month = sorted(ques_agg_all_pivot['Activity_month'].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', 'Activity_month'])
    ques_agg_all_pivot = (
        ques_agg_all_pivot.set_index(['product_code', 'Activity_month'])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    for c in  ques_agg_all_pivot.columns:
        if c not in ['product_code', 'Activity_month']:
            ques_agg_all_pivot.rename(columns={c: f'Avg Question Answered_{c}'}, inplace=True)

    ques_agg_all_pivot['Activity_month'] = pd.to_datetime(ques_agg_all_pivot['Activity_month'], format='%m').dt.strftime('%b')
    return ques_agg_all_pivot
monthwise_ques = get_monthwise_question()
def get_monthwise_pt():
    pt_agg_all = (
            aamc_kfl_data.groupby(['product_code', 'Activity_month', 'Activity_year' ])
            .agg(sequence_name=('sequence_name', 'count'))
            .reset_index()
        )
    pt_agg_all_pivot = pt_agg_all.pivot_table(
            index=['product_code', 'Activity_month'],
            columns='Activity_year',
            values=['sequence_name'],
            fill_value=0
        )
    pt_agg_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in pt_agg_all_pivot.columns]
    pt_agg_all_pivot = pt_agg_all_pivot.reset_index()
    pt_agg_all_pivot.columns.name = None  # Remove the columns name
    pt_agg_all_pivot['Activity_month'] = pt_agg_all_pivot['Activity_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_prod = aamc_kfl_data['product_code'].unique()
    all_month = sorted(pt_agg_all_pivot['Activity_month'].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', 'Activity_month'])
    pt_agg_all_pivot = (
        pt_agg_all_pivot.set_index(['product_code', 'Activity_month'])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )

    for yrs in years:
        pt_agg_all_pivot[f'kbs_enrollment_id_{yrs}'] = monthwise_act[f'Avg_Activity_kbs_enrollment_id_{yrs}']
    pt_agg_all_pivot['Activity_month'] = pd.to_datetime(pt_agg_all_pivot['Activity_month'], format='%m').dt.strftime('%b')

    for c in  pt_agg_all_pivot.columns:
        if c not in ['product_code', 'Activity_month']:
            pt_agg_all_pivot.rename(columns={c: f'Avg Practice Test_{c}'}, inplace=True)
   
    return pt_agg_all_pivot

monthwise_act_ques_pt = pd.concat([monthwise_act , monthwise_ques , get_monthwise_pt()] , axis=1)
load_to_gsheets(monthwise_act_ques_pt, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Monthwise Activity_Questions_PT_metrics')

## Activities Completed Trends 30,60,90 days

In [69]:
Act_data_cleaned_all_req = Act_data_cleaned_all[Act_data_cleaned_all['ESD_year'].isin(years)]
today = pd.Timestamp.today().normalize()
Act_data_cleaned_all_req['days_to_start'] = (today - Act_data_cleaned_all_req['enroll_start_date']).dt.days
Act_data_cleaned_all_req['Act_to_start'] = (Act_data_cleaned_all_req['date_created']- Act_data_cleaned_all_req['enroll_start_date']).dt.days
def get_activity_completion_trends(df_in,agg_col,agg_type,days_criteria , pivot_type):
    elig_ids = df_in[df_in['days_to_start'] >= days_criteria]['kbs_enrollment_id'].unique()
    data_yrs_all = Act_data_cleaned_all_req[(Act_data_cleaned_all_req['kbs_enrollment_id'].isin(elig_ids)) & (Act_data_cleaned_all_req['Act_to_start'] <= days_criteria)]
    act_comp_agg_all = (
            data_yrs_all.groupby(['product_code', 'ESD_month', 'ESD_year' ])
            .agg({'kbs_enrollment_id': 'nunique', agg_col: agg_type})
            .reset_index()
        )
    act_comp_agg_all_pivot = act_comp_agg_all.pivot_table(
            index=['product_code', 'ESD_month'],
            columns='ESD_year',
            values=[agg_col, 'kbs_enrollment_id'],
            fill_value=0
        )
    act_comp_agg_all_pivot.columns = [f"{col1}_{col2}" for col1, col2 in act_comp_agg_all_pivot.columns]
    act_comp_agg_all_pivot = act_comp_agg_all_pivot.reset_index()
    act_comp_agg_all_pivot.columns.name = None  # Remove the columns name
    act_comp_agg_all_pivot['ESD_month'] = act_comp_agg_all_pivot['ESD_month'].astype(int)
    # Ensure all months and prod_codes appear (even if missing)
    all_month = sorted(act_comp_agg_all_pivot['ESD_month'].unique())
    complete_index = pd.MultiIndex.from_product([all_prod, all_month], names=['product_code', 'ESD_month'])
    act_comp_agg_all_pivot = (
        act_comp_agg_all_pivot.set_index(['product_code', 'ESD_month'])
                .reindex(complete_index, fill_value=0)
                .reset_index()
    )
    col=[]
    for c in act_comp_agg_all_pivot.columns:
        if c in ['product_code','ESD_month']:
            col.append(c)
        else:
            col.append(f'{pivot_type}_{days_criteria}_days_{c}')

    act_comp_agg_all_pivot.columns = col
    act_comp_agg_all_pivot['ESD_month'] = pd.to_datetime(act_comp_agg_all_pivot['ESD_month'], format='%m').dt.strftime('%b')
    return act_comp_agg_all_pivot


activity_completion_30 = get_activity_completion_trends(Act_data_cleaned_all_req,'sequence_title','count',30,'Activity')
activity_completion_60 = get_activity_completion_trends(Act_data_cleaned_all_req,'sequence_title','count',60,'Activity')
activity_completion_90 = get_activity_completion_trends(Act_data_cleaned_all_req,'sequence_title','count',90,'Activity')
activity_completion_30_60_90 = pd.concat([activity_completion_30 , activity_completion_60 , activity_completion_90] , axis=1)
load_to_gsheets(activity_completion_30_60_90, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Activity_Completion_Trends(30_60_90_days)_metrics')
    


In [76]:
def get_req_filtered_df_act_ques_pt(df_in, prod_code , agg_month,param_col,agg_col):
    filtered_df = df_in[df_in['product_code'].isin(prod_code)]
    final_df = pd.DataFrame(df_in[agg_month].unique(), columns=[agg_month])
    for yrs in years:
        agg_df = filtered_df.groupby([agg_month]).agg({
            f'{param_col}_kbs_enrollment_id_{yrs}':'sum',
            f'{param_col}_{agg_col}_{yrs}':'sum',
        }).reset_index()
        agg_df[f'{param_col}_{yrs}'] = agg_df[f'{param_col}_{agg_col}_{yrs}'] / agg_df[f'{param_col}_kbs_enrollment_id_{yrs}']
        final_df = pd.merge(final_df, agg_df, on=agg_month, how='left')
    final_df = final_df.fillna(0)
    final_df = final_df.replace(0,'')
    # print(param_col , final_df.columns)
    return final_df

test_30_days = get_req_filtered_df_act_ques_pt(activity_completion_30_60_90.iloc[:, 0:8] , all_product_codes , 'ESD_month' , 'Activity_30_days' , 'sequence_title')
test_30_days

,ESD_month,Activity_30_days_kbs_enrollment_id_2023,Activity_30_days_sequence_title_2023,Activity_30_days_2023,Activity_30_days_kbs_enrollment_id_2024,Activity_30_days_sequence_title_2024,Activity_30_days_2024,Activity_30_days_kbs_enrollment_id_2025,Activity_30_days_sequence_title_2025,Activity_30_days_2025
0,Jan,1500.0,94177.0,62.784667,1677.0,148467.0,88.531306,1803.0,184956.0,102.582363
1,Feb,607.0,33172.0,54.649094,863.0,60659.0,70.288528,864.0,77540.0,89.74537
2,Mar,432.0,24980.0,57.824074,589.0,40619.0,68.962649,961.0,65978.0,68.655567
3,Apr,448.0,33334.0,74.406250,642.0,58887.0,91.724299,870.0,74526.0,85.662069
4,May,1586.0,150413.0,94.837957,1929.0,199745.0,103.548471,2331.0,255261.0,109.507079
5,Jun,951.0,83155.0,87.439537,995.0,85535.0,85.964824,1332.0,134856.0,101.243243
6,Jul,649.0,47407.0,73.046225,649.0,47897.0,73.801233,843.0,68071.0,80.748517
7,Aug,871.0,53416.0,61.327210,913.0,62083.0,67.998905,1171.0,88575.0,75.640478
8,Sep,811.0,51750.0,63.810111,1050.0,67524.0,64.308571,1010.0,80627.0,79.828713
9,Oct,887.0,52030.0,58.658399,916.0,66881.0,73.014192,,,


In [70]:
ques_completion_30 = get_activity_completion_trends(Act_data_cleaned_all_req,'total_scored_items_answered','sum',30,'Ques_Ans')
ques_completion_60 = get_activity_completion_trends(Act_data_cleaned_all_req,'total_scored_items_answered','sum',60,'Ques_Ans')
ques_completion_90 = get_activity_completion_trends(Act_data_cleaned_all_req,'total_scored_items_answered','sum',90,'Ques_Ans')

ques_completion_30_60_90 = pd.concat([ques_completion_30 , ques_completion_60 , ques_completion_90] , axis=1)
load_to_gsheets(ques_completion_30_60_90, '1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4' , 'Ques_Ans_Completion_Trends(30_60_90_days)_metrics')

In [91]:
# check_df = read_from_gsheets_area('1wZmKXpk0nXaQb1aNvzjb-PFkVlpqvOc8Lj4_o49oso4','KPIs_sheet_tll_PM','A2','AF27')
check_df.iloc[:,7:13]

,product_code,total_sequence_name_24,total_eid_24,total_sequence_name_25,total_eid_25,Parameter
0,MCATVL9,116723,745,83200,546,Avg Activity / User
1,MCATVL,815953,4676,990858,4730,Avg Activity / User
2,MCATOLX,812950,4306,889393,4203,Avg Activity / User
3,MCATOLX12,160493,1328,228846,1923,Avg Activity / User
4,MCATVLX,46963,262,221260,990,Avg Activity / User
5,MCAT,105260,889,100543,729,Avg Activity / User
6,MCATVL12,446104,4273,592578,5669,Avg Activity / User
7,MCATOLX9,153048,853,92561,513,Avg Activity / User
8,MCAT12,69059,621,56726,476,Avg Activity / User
9,MCTEXVL12,15008,180,25675,230,Avg Activity / User
